In [1]:
import header

from multiprocessing.pool import ThreadPool
from pathlib import Path

import pandas as pd
import numpy as np
import os
import requests
import tldextract
from v0.ai import embedding_model
from v0.models import Image


FILE = Path('data/gcc/validation.tsv')

if not FILE.is_file():
    raise FileNotFoundError(f'{FILE} not found')


DJANGO SETTINGS IN DEBUG
[sentence_transformers.SentenceTransformer] 2022-05-30 21:17:26,178 INFO MainThread [SentenceTransformer:__init__:60] | Load pretrained SentenceTransformer: all-mpnet-base-v2
[sentence_transformers.SentenceTransformer] 2022-05-30 21:17:34,208 INFO MainThread [SentenceTransformer:__init__:97] | Use pytorch device: cpu


In [2]:
images = pd.read_csv(FILE, sep='\t', header=None, names=['label', 'url'])
print(images.head())

# filter to ones we havent done yet
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
done_urls = list(Image.objects.all().values_list('url', flat=True))
print(len(done_urls))

images = images.drop(images[images.url.isin(done_urls)].index)
print(len(images))

                                               label  \
0      author : a life in photography -- in pictures   
1            an angler fishes river on a snowy day .   
2  photograph of the sign being repaired by brave...   
3  the player staring intently at a computer scre...   
4  globes : the green 3d person carrying in hands...   

                                                 url  
0  https://i.pinimg.com/736x/66/01/6c/66016c3ba27...  
1  http://www.standard.net/image/2015/02/04/800x_...  
2  http://indianapolis-photos.funcityfinder.com/f...  
3  http://www.abc.net.au/news/image/9066492-3x2-7...  
4  https://www.featurepics.com/StockImage/2009031...  
10599
5241


In [3]:
# embeddings
embeds = np.array(embedding_model.model.encode(images['label'].to_list())).astype(np.float32)
images['embed'] = embeds.tolist()

Batches:   0%|          | 0/164 [00:00<?, ?it/s]

In [9]:

def process_image(i_image):
    i, image = i_image
    try:
        image_model = Image()
        image_model.url = image['url']
        image_model.description = image['label']
        image_model.embedding_all_mpnet_base_v2 = image['embed']
        image_model.domain = tldextract.extract(image['url']).domain
        image_model.provider = Image.providers.GCC_DATASET

        # validate the image is alive
        r = requests.head(image['url'], timeout=1)
        if r.status_code != 200:
            image_model.url_alive = False
        image_model.save()
        return
    except Exception as e:
        print(e)

# for i, image in images.iterrows(): 
#     process_image((i, image))
#     print(i)
pool = ThreadPool(processes=16)
pool.map(process_image, images.iterrows())

HTTPConnectionPool(host='www.bawaba-turkey.com', port=80): Max retries exceeded with url: /images/19.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x2e0b28370>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
duplicate key value violates unique constraint "v0_image_url_key"
DETAIL:  Key (url)=(https://lookaside.fbsbx.com/lookaside/crawler/media/?media_id=1655545241162868) already exists.

HTTPConnectionPool(host='www.brianmassa.org', port=80): Max retries exceeded with url: /images/Dubrovnik_31-Shopped.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x2e0b3cfd0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
HTTPConnectionPool(host='photos.mycapture.com', port=80): Max retries exceeded with url: /DYTN/1480715/42147902E.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x2eeb7ed60>: Failed to est